In [1]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd

In [2]:
title = "Data Analytics"
location = "Colombia"

In [3]:
list_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=analista%2Bde%2Bdatos&location=colombia&start=50" #editar link 
response = requests.get(list_url)

#https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=analista%2Bde%2Bdatos&location=colombia&geoId=&trk=public_jobs_jobs-search-bar_search-submit&start=25

list_data = response.text 
list_soup = BeautifulSoup(list_data, "html.parser")
page_jobs = list_soup.find_all("li")

In [ ]:
# Parámetros de búsqueda
title = "analista de datos"
location = "colombia"

# URL base para la paginación
base_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search"
id_list = []

# Lógica para paginar y obtener más IDs
for start in range(0, 200, 25):  # Cambiar 200 según el número máximo deseado
    list_url = f"{base_url}?keywords={title.replace(' ', '%2B')}&location={location}&start={start}"
    response = requests.get(list_url)
    if response.status_code != 200:
        print(f"Error al obtener datos de la página con inicio {start}")
        break
    list_data = response.text
    list_soup = BeautifulSoup(list_data, "html.parser")
    page_jobs = list_soup.find_all("li")
    
    # Extraer IDs de los trabajos
    for job in page_jobs:
        try:
            base_card_div = job.find("div", {"class": "base-card"})
            if base_card_div:
                job_id = base_card_div.get("data-entity-urn").split(":")[3]
                id_list.append(job_id)
        except AttributeError:
            continue

# Limitar el número de IDs a procesar si es necesario
id_list = list(set(id_list))  # Eliminar duplicados
print(f"Se encontraron {len(id_list)} IDs únicos.")

# Lista para almacenar información de trabajos
job_list = []

# Obtener información de cada trabajo
for job_id in id_list:
    # URLs de la vacante
    job_url_api = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    job_url_decent = f"https://www.linkedin.com/jobs/view/{job_id}"  # URL limpia
    
    # Solicitar datos del trabajo
    job_response = requests.get(job_url_api)
    if job_response.status_code != 200:
        continue  # Saltar si hay error en la solicitud
    
    # Parsear datos del trabajo
    job_soup = BeautifulSoup(job_response.text, "html.parser")
    job_post = {"job_url": job_url_decent}  # Incluir la URL limpia
    
    # Extraer detalles del trabajo
    try:
        job_post["job_title"] = job_soup.find("h2", {"class": "topcard__title"}).text.strip()
    except AttributeError:
        job_post["job_title"] = None

    try:
        job_post["company_name"] = job_soup.find("a", {"class": "topcard__org-name-link"}).text.strip()
    except AttributeError:
        job_post["company_name"] = None

    try:
        job_post["nivel"] = job_soup.find("span", {"class": "description__job-criteria-text"}).text.strip()
    except AttributeError:
        job_post["nivel"] = None


# intentar sacar el nivel en 4 
    try:
        criterios = job_soup.find_all("span", {"class": "description__job-criteria-text"})
        if len(criterios) >= 4:
            job_post["nivel_4"] = criterios[3].text.strip()
        else:
            job_post["nivel_4"] = None
    except AttributeError:
        job_post["nivel_4"] = None

"""try:
    job_post["nivel"] = job_soup.find_all("span", {"class": "description__job-criteria-text"})[3].text.strip()
except (AttributeError, IndexError):
    job_post["nivel"] = None"""

    try:
        job_post["time_posted"] = job_soup.find("span", {"class": "posted-time-ago__text"}).text.strip()
    except AttributeError:
        job_post["time_posted"] = None

    try:
        job_post["num_applicants"] = job_soup.find("span", {"class": "num-applicants__caption"}).text.strip()
    except AttributeError:
        job_post["num_applicants"] = None

    # Detalles del reclutador
    try:
        recruiter_tag = job_soup.find("a", {"class": "base-card__full-link"})
        job_post["recruiter_name"] = recruiter_tag.text.strip()
        job_post["recruiter_link"] = recruiter_tag.get("href")
    except AttributeError:
        job_post["recruiter_name"] = None
        job_post["recruiter_link"] = None
    
    # Dentro del loop para obtener información de cada trabajo

# Extraer el enlace de la empresa
    try:
        company_tag = job_soup.find("a", {"class": "topcard__org-name-link"})
        job_post["company_name"] = company_tag.text.strip()
        job_post["company_link"] = company_tag.get("href")  # Extraer enlace de la empresa
    except AttributeError:
        job_post["company_name"] = None
        job_post["company_link"] = None



    # Agregar el trabajo a la lista
    job_list.append(job_post)

# Ordenar trabajos por fecha de publicación y limitar a 30
job_list = sorted(job_list, key=lambda x: x["time_posted"] or "", reverse=True)[:30]

# Guardar los resultados en un archivo Excel
jobs_df = pd.DataFrame(job_list)
jobs_df.to_excel("LinkedIn_Recent_Jobs.xlsx", index=False, engine="openpyxl")

print("Se han guardado los resultados en LinkedIn_Recent_Jobs.xlsx")


Se encontraron 74 IDs únicos.
Se han guardado los resultados en LinkedIn_Recent_Jobs.xlsx


In [19]:
# Create a pandas DataFrame using the list of job dictionaries 'job_list'
jobs_df = pd.DataFrame(job_list)
jobs_df

,job_url,job_title,company_name,nivel,nivel_4,time_posted,num_applicants,recruiter_name,recruiter_link,company_link
0,https://www.linkedin.com/jobs/view/3944267877,Analista de datos,Alianza Temporales SAS,Mid-Senior level,Human Resources Services,7 months ago,None,None,None,https://co.linkedin.com/company/alianza-tempor...
1,https://www.linkedin.com/jobs/view/3971955160,BI Analyst,GomezLee Marketing,Mid-Senior level,Advertising Services,7 months ago,None,None,None,https://do.linkedin.com/company/gomezlee-marke...
2,https://www.linkedin.com/jobs/view/3934323161,Anlista logistico / analista de datos,Alianza Temporales SAS,Mid-Senior level,Human Resources Services,7 months ago,159 applicants,None,None,https://co.linkedin.com/company/alianza-tempor...
3,https://www.linkedin.com/jobs/view/4125309289,Data Engineer,Ideaware,Mid-Senior level,Internet Publishing,6 hours ago,None,None,None,https://www.linkedin.com/company/ideaware-co?t...
4,https://www.linkedin.com/jobs/view/4119095307,Data Analyst,CRISIL Limited,Entry level,Financial Services,6 days ago,None,Camila Cabuya Rodríguez,https://co.linkedin.com/in/camila-cabuya/en,https://in.linkedin.com/company/crisil?trk=pub...
5,https://www.linkedin.com/jobs/view/4117952930,Work From Home Talent Data Analyst,L.A. Head Hunter,Mid-Senior level,Staffing and Recruiting and Consumer Services,6 days ago,None,Andrea Baptista,https://www.linkedin.com/in/andrea-baptista-39...,https://mx.linkedin.com/company/la-head-hunter...
6,https://www.linkedin.com/jobs/view/3980650937,Reporting Junior,GroupCOS,Not Applicable,IT Services and IT Consulting,5 months ago,69 applicants,None,None,https://co.linkedin.com/company/colombianoutso...
7,https://www.linkedin.com/jobs/view/4120631804,Analytics Engineer,Nubank,Entry level,Financial Services,5 days ago,None,None,None,https://br.linkedin.com/company/nubank?trk=pub...
8,https://www.linkedin.com/jobs/view/4117129571,Data Analyst,Cabify,Mid-Senior level,"Technology, Information and Internet",5 days ago,None,None,None,https://es.linkedin.com/company/cabify?trk=pub...
9,https://www.linkedin.com/jobs/view/4125428525,Analista de Inteligencia de Negocio - 26530,Corbeta Textiles,Entry level,Executive Offices,4 hours ago,None,None,None,https://co.linkedin.com/company/corbeta-textil...
